In [2]:
from networkit import *
import glob
from tqdm import tqdm_notebook
import networkit as nx
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
import datetime
import pickle
from glob import glob
from datetime import datetime


In [3]:
nx.setNumberOfThreads(20)
nx.setLogLevel("TRACE")

In [4]:
with open('../user_details.json') as fp:
    dict_users_original = json.load(fp)
with open('../Networks/interaction_network_per_month.json', 'rb') as fp:
    data = pickle.load(fp)
with open('../Networks/kcore_temportal.json', 'rb') as fp:
    monthwise_kcore_dict = pickle.load(fp)  
with open('../Networks/map_dict_kcore.json', 'rb') as fp:
    map_dict_kcore = pickle.load(fp)  
    
df_user_label=pd.read_csv('../Results/user_labelling.csv')
all_normal_dict={key:1 for key in list(df_user_label[df_user_label['label']=='normal']['user_key'])}
# df_matched=pd.read_csv('../Results/matched.csv')
df_matched=pd.read_csv('../Results/matched_fear_hate.csv')


In [5]:
with open('../Networks/interaction_network_per_month.json', 'rb') as fp:
    interaction_data = pickle.load(fp)


In [6]:
with open('../Networks/betweeness_centrality.json', 'rb') as fp:
    betweenness_data = pickle.load(fp)
    
with open('../Networks/eigen_vector_centrality.json', 'rb') as fp:
    eigen_data = pickle.load(fp)


In [24]:
df_betweenness=pd.DataFrame(betweenness_data.values())

In [25]:
df_users_timeline=pd.read_csv('../Results/kcore_analysis.csv')


In [9]:
file = '../../../../GAB_data/FollowersFollowing/Follow_Following_June2018.json'

In [10]:
threshold={'hatespeech':0.9,'fearspeech':0.7,'normal':0.7}
dict_labels={'hatespeech':2,'fearspeech':1,'normal':0}
reverse_dict_labels={dict_labels[key]:key for key in dict_labels.keys()}

In [12]:
dict_users={}
dict_users_ambi={}
files = sorted(glob('../../Gab_Data/new_features_old_gab/gab_fear_hate_features*.pickle'))


dict_dates={}
for file in tqdm_notebook(files,total=len(files)):
    
    with open(file, 'rb') as handle:
        Gab_keyword_match = pickle.load(handle)
    
    if('predicted_probab' not in Gab_keyword_match[1].keys()):
        continue
    for element in Gab_keyword_match:
        
        
        try:
            dt = datetime.fromisoformat(element['post_create_time'])
            key_new=str(dt.month)+'/'+str(dt.year)
        except KeyError:
            continue

    
        
        labels=[]
        for i in range(len(element['predicted_probab'])):
            if(element['predicted_probab'][i]>threshold[reverse_dict_labels[i]]):
                labels.append(reverse_dict_labels[i])
        
        labels_ambi=[]
        for i in range(len(element['predicted_probab'])):
            if(element['predicted_probab'][i]>0.5):
                labels_ambi.append(reverse_dict_labels[i])
        
        
        try:
            for label in labels:
                dict_users[element['username']][label]+=1
            dict_users[element['username']]['total']+=1
        except KeyError:
            try:
                dict_users[element['username']]={'fearspeech':0,'hatespeech':0,'normal':0,'total':0}
                for label in labels:
                    dict_users[element['username']][label]+=1
                dict_users[element['username']]['total']+=1

            except KeyError:
                pass
#         try:
#             for label in labels:
#                 dict_users_ambi[element['username']][label]+=1
#             dict_users_ambi[element['username']]['total']+=1
#         except KeyError:
#             dict_users_ambi[element['username']]={'fearspeech':0,'hatespeech':0,'normal':0,'total':0}
#             for label in labels:
#                 dict_users_ambi[element['username']][label]+=1
#             dict_users_ambi[element['username']]['total']+=1

            

<ipython-input-12-087242fa3de8>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for file in tqdm_notebook(files,total=len(files)):


  0%|          | 0/212 [00:00<?, ?it/s]

In [13]:
dict_users_filter={}
max_fear=0
max_hate=0
for key in dict_users.keys():
    user=dict_users[key]
    total=user['fearspeech']+user['hatespeech']
    
    if(user['fearspeech']>max_fear):
        max_fear=user['fearspeech']
    
    if(user['hatespeech']>max_hate):
        max_hate=user['hatespeech']
    
    if(total>=10):
        dict_users_filter[key]=user
        dict_users_filter[key]['name']=key


In [14]:
df_final_users=pd.DataFrame(dict_users_filter).transpose()

In [15]:
df_final_users.loc[list(df_matched.control)]

,fearspeech,hatespeech,normal,total,name
libtardOZ,105,33,428,871,libtardOZ
libtardOZ,105,33,428,871,libtardOZ
EyesBlue71,196,6,690,1340,EyesBlue71
1488Goy,303,130,651,2090,1488Goy
ArtificeCubed,93,14,2697,3659,ArtificeCubed
...,...,...,...,...,...
ramenanne,116,43,265,853,ramenanne
Obbop,146,20,87,389,Obbop
Obbop,146,20,87,389,Obbop
Obbop,146,20,87,389,Obbop


In [ ]:
df_final_users.to_csv('../Results/user_distribution.csv',index=False)

### Final analysis

In [16]:
list_bd=[]
list_ev=[]
list_followers=[]
list_following=[]


for index,row in df_final_users.iterrows():
    list_bd.append(betweenness_data[index])
    list_ev.append(eigen_data[index])
    list_following.append(dict_users_original[index]['followers_count'])
    list_followers.append(dict_users_original[index]['followings_count'])


In [17]:
df_final_users['eigen_vector']=list_ev
df_final_users['betweeness']=list_bd
df_final_users['followers']=list_followers
df_final_users['following']=list_following

In [20]:
df_final_users

,fearspeech,hatespeech,normal,total,name,eigen_vector,betweeness,followers,following
radiofan2,47,26,1716,2287,radiofan2,6.149054e-03,0.000000,967,1103
kaser100,306,42,4862,7319,kaser100,1.551704e-02,0.000055,2082,1904
DeplorableRick,18,19,760,1241,DeplorableRick,5.291231e-03,0.000000,800,978
kingdomMan,89,4,1172,1969,kingdomMan,2.564034e-03,0.000000,536,1043
Whicket,490,108,16161,21862,Whicket,1.562968e-02,0.000041,2059,1767
...,...,...,...,...,...,...,...,...,...
WeAreAllTommyRobinson,8,2,9,45,WeAreAllTommyRobinson,0.000000e+00,0.000000,0,21
Luther2,7,23,8,62,Luther2,0.000000e+00,0.000000,0,5
anhero23,3,10,74,157,anhero23,1.630122e-03,0.000000,185,256
PewDieNazi,0,15,12,31,PewDieNazi,3.201372e-05,0.000014,6,5


In [18]:
# df_final_users['fearspeech']=df_final_users['fearspeech']/df_final_users['total']
# df_final_users['hatespeech']=df_final_users['hatespeech']/df_final_users['total']
dict_users_original[index]

{'number_of_posts': 64, 'followers_count': 6, 'followings_count': 1}

In [23]:
df_90

NameError: name 'df_90' is not defined

In [21]:
following_follower_ratio=[]
for index,row in df_90.iterrows():
    if(row['followers']>10):
        
        following_follower_ratio.append(row['following']/row['followers'])

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/punyajoy/.conda/envs/punyajoy_gpu/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-21-147c0d4b925b>", line 2, in <module>
    for index,row in df_90.iterrows():
NameError: name 'df_90' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/punyajoy/.conda/envs/punyajoy_gpu/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2045, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/punyajoy/.conda/envs/punyajoy_gpu/lib/python3.8/site-packages/IPython/core/ultratb.py", line 1170, in get_records
    return _fixed_getinnerframes(etb, number_of_li

TypeError: object of type 'NoneType' has no len()

In [ ]:
df_final_users.head(5)

In [ ]:
count=1
for ratio in following_follower_ratio:
    if(ratio>10):
        count+=1

count/len(df_final_users)

In [ ]:
time_to_reach_list=[]
velocity=[]
for index,row in tqdm_notebook(df_final_users.iterrows(),total=len(df_final_users)):
    try:
        time=df_users_timeline[df_users_timeline['user_key']==index]['time_to_reach'].iloc[0]
        vel=df_users_timeline[df_users_timeline['user_key']==index]['velocity'].iloc[0]
    except IndexError:
        time=-1
        vel=-1
    time_to_reach_list.append(time)
    velocity.append(vel)
    

In [ ]:
df_final_users['time_to_reach']=time_to_reach_list
df_final_users['velocity']=velocity

In [ ]:
df_final_users.head()

### Follower distribution

In [ ]:
import networkx as nx
 

def get_graph_features(file):
    map_dict={}
    follower_dict={}
    count=0
    with open(file) as fp:
        dict_follower_followship=json.load(fp)

    for user_key in tqdm_notebook(dict_follower_followship.keys()):
        try:
            temp=map_dict[user_key]
        except KeyError:
            map_dict[user_key]=count
            count+=1



        for user_follower in dict_follower_followship[user_key]['follower']:
            try:
                temp=map_dict[user_follower]
            except KeyError:
                map_dict[user_follower]=count
                count+=1

            try:
                follower_dict[map_dict[user_key]][map_dict[user_follower]]=1
            except KeyError:
                follower_dict[map_dict[user_key]]={}
                follower_dict[map_dict[user_key]][map_dict[user_follower]]=1

    reverse_map_dict={value:key for key,value in map_dict.items()}

    G = nx.DiGraph()
    for user_key in follower_dict.keys():
        for follower_key in follower_dict[user_key].keys():
            G.add_edge(user_key, follower_key)
            
    return G, map_dict, reverse_map_dict



In [ ]:
file='../../../../GAB_data/FollowersFollowing/Follow_Following_June2018.json'
G, map_dict, reverse_map_dict=get_graph_features(file)

In [ ]:
len(G.nodes())

In [ ]:
normal_followers=[]
for index,row in tqdm_notebook(df_final_users.iterrows(),total=len(df_final_users)):
    try:
        succesors=list(G.successors(map_dict[index]))
        normal=0
        total=0
        for suc in succesors:
            user_suc_name=reverse_map_dict[suc]
            try:
                temp=all_normal_dict[user_suc_name]
                normal+=1
            except KeyError:
                pass
            total+=1
        if(total>0):
            normal_followers.append(normal/total)
        else:
            normal_followers.append(0)
    except (nx.NetworkXError, KeyError) as e:
        normal_followers.append(-1)
        pass
        

In [ ]:
df_final_users['normal_followers']=normal_followers

In [ ]:
df_final_users

### Interactions 

In [ ]:
interaction_dict={}
for month_year in tqdm_notebook(interaction_data.keys()):
    user_data=interaction_data[month_year]
    for key in user_data.keys():
        try:
            interaction_dict[key[0]][key[1]]+=user_data[key]['replies']
        except KeyError:
            try:
                interaction_dict[key[0]][key[1]]=user_data[key]['replies']
            except KeyError:
                interaction_dict[key[0]]={}
                interaction_dict[key[0]][key[1]]=user_data[key]['replies']
            

In [ ]:
edges_count=0
for month_year in tqdm_notebook(interaction_data.keys()):
    user_data=interaction_data[month_year]
    for key in user_data.keys():
        if(user_data[key]['replies']>0):
            edges_count+=1
        if(user_data[key]['reposts']>0):
            edges_count+=1
        if(user_data[key]['mentions']>0):
            edges_count+=1
        
        
        

In [ ]:
edges_count

In [ ]:
import networkx
G = networkx.DiGraph()
for user_key in tqdm_notebook(interaction_dict.keys()):
    for inter_user_key in interaction_dict[user_key].keys():
        if(user_key==inter_user_key):
            continue
        if(interaction_dict[user_key][inter_user_key]>0):
            G.add_edge(inter_user_key,user_key,weight=interaction_dict[user_key][inter_user_key])


In [ ]:
len(G.edges())


In [ ]:
normal_reposters=[]
total_normal_reposts=[]
for index,row in tqdm_notebook(df_final_users.iterrows(),total=len(df_final_users)):
    try:
        succesors=list(G.successors(index))
        normal=0
        total_normal=0
        for user_suc_name in succesors:
            weight=G[index][user_suc_name]["weight"]
            try:
                temp=all_normal_dict[user_suc_name]
                normal+=weight
                total_normal+=1
            except KeyError:
                pass
            
        normal_reposters.append(total_normal)
        total_normal_reposts.append(normal)
            
    
    except (nx.NetworkXError, KeyError) as e:
        total_normal_reposts.append(-1)
        normal_reposters.append(-1)
        pass
    


In [ ]:
df_final_users['normal_reposters']=normal_reposters
df_final_users['total_normal_reposts']=total_normal_reposts

In [ ]:
df_final_users['normal_mentions']=normal_reposters
df_final_users['total_normal_mentions']=total_normal_reposts


In [ ]:
df_final_users['normal_repliers']=normal_reposters
df_final_users['total_normal_replies']=total_normal_reposts


In [ ]:
df_final_users.columns

### Stats model

In [ ]:
import statsmodels.api as sm
import numpy as np

In [ ]:
str_column='normal_followers'
linear_model_str=str_column+" ~fearspeech+hatespeech+total"

df_final_users_temp=df_final_users[['fearspeech','hatespeech','total',str_column]]
#normalized_df=(df_final_users_temp-df_final_users_temp.min())/(df_final_users_temp.max()-df_final_users_temp.min())
print(np.mean(df_final_users[str_column]))
model_lin = sm.OLS.from_formula(linear_model_str,data=df_final_users_temp)
result_lin = model_lin.fit()
result_lin.summary()

In [ ]:
str_column='normal_followers'
linear_model_str=str_column+" ~ fearspeech+hatespeech+total"
df_final_users_temp=df_final_users[df_final_users[str_column]!=-1]
print(np.mean(df_final_users_temp[str_column]),len(df_final_users_temp))
#model_lin = sm.OLS.from_formula(linear_model_str,data=df_final_users_temp[['fearspeech','hatespeech','total',str_column]])
model_lin = sm.GLM.from_formula(linear_model_str,data=df_final_users_temp[['fearspeech','hatespeech','total',str_column]],family=sm.families.NegativeBinomial())
result_lin = model_lin.fit()
result_lin.summary()

## Interaction network

In [ ]:
import numpy as np

In [ ]:
np.var(df_final_users['followers'])


In [ ]:
df_90_hate = df_final_users.loc[list(df_matched.treatment)]
df_90_hate['label']='H'

In [ ]:
df_90_hate

In [ ]:
df_90_fear = df_final_users.loc[list(df_matched.control)]
df_90_fear['label']='F'

In [ ]:
df_90_fear

In [ ]:
df_90=pd.concat([df_90_fear,df_90_hate],ignore_index=False)

In [ ]:
df_90[df_90['label']=='F']['total'].sum()

In [ ]:
df_90_hate.time_to_reach.describe()

In [ ]:
import numpy as np
matplotlib.rcParams['figure.figsize'] = (3, 3)
sns.set(context='paper',style='whitegrid',font_scale=1.5)

# sns.set_theme(style="whitegrid")

# tips = sns.load_dataset("tips")
ax = sns.barplot(x="label",y="betweeness", data=df_90, palette=['red','orange'])
#plt.show()
path='../Results/betweenness_centrality.pdf'
plt.savefig(path,bbox_inches='tight',dpi=400)

In [ ]:
import numpy as np
matplotlib.rcParams['figure.figsize'] = (3, 3)
sns.set(context='paper',style='whitegrid',font_scale=1.5)

# sns.set_theme(style="whitegrid")

# tips = sns.load_dataset("tips")
ax = sns.barplot(x="label",y="eigen_vector", data=df_90, palette=['red','orange'])
#plt.show()
path='../Results/eigen_vector_centrality.pdf'
plt.savefig(path,bbox_inches='tight',dpi=400)

In [ ]:
import numpy as np
matplotlib.rcParams['figure.figsize'] = (3, 3)
sns.set(context='paper',style='whitegrid',font_scale=1.5)

# sns.set_theme(style="whitegrid")

# tips = sns.load_dataset("tips")
ax = sns.barplot(x="label",y="followers", data=df_90, palette=['red','orange'])
#plt.show()
path='../Results/followers.pdf'
plt.savefig(path,bbox_inches='tight',dpi=400)

In [ ]:
df_90

In [ ]:
import numpy as np
matplotlib.rcParams['figure.figsize'] = (3, 3)
sns.set(context='paper',style='whitegrid',font_scale=1.5)

# sns.set_theme(style="whitegrid")

# tips = sns.load_dataset("tips")
ax = sns.barplot(x="label",y="following", data=df_90, palette=['red','orange'])
#plt.show()
path='../Results/followings.pdf'
plt.savefig(path,bbox_inches='tight',dpi=400)

In [ ]:
df_90.to_csv('../Results/top_10_percentile.csv',index=False)

In [ ]:
import numpy as np
matplotlib.rcParams['figure.figsize'] = (3, 3)
sns.set(context='paper',style='whitegrid',font_scale=1.5)

# sns.set_theme(style="whitegrid")

# tips = sns.load_dataset("tips")
ax = sns.barplot(x="label",y="normal_followers", data=df_90, palette=['red','orange'])
#plt.show()
path='../Results/normal_followers.pdf'
plt.savefig(path,bbox_inches='tight',dpi=400)

In [ ]:
df_90['normal_followers']=df_90['normal_followers']/(df_90['followers']+1e-15)

In [ ]:
import plotly.graph_objects as go
import urllib, json

url = 'https://raw.githubusercontent.com/plotly/plotly.js/master/test/image/mocks/sankey_energy.json'
response = urllib.request.urlopen(url)
data = json.loads(response.read())
colours=data['data'][0]['node']['color']

def get_sources_target_val(df):
    dict_tuples={}
    for i in range(0,len(months_list)-1):
        key=tuple([months_list[i],months_list[i+1]])
        dict_tuples[key]={}
        wide_df = df[[months_list[i],months_list[i+1],'freq']].groupby([months_list[i],months_list[i+1]], as_index=False).sum()
        columns=list(wide_df.columns)[0:3]
        for index,row in wide_df.iterrows():
            dict_tuples[key][tuple([row[columns[0]],row[columns[1]]])]=row['freq']
    
    sources=[]
    target=[]
    values=[]
    colour_values=[]
    i=0
    for key in dict_tuples.keys():
        for element in sorted(list(dict_tuples[key].keys())):
            print(element)
            source_index=label[key[0]+'-'+str(element[0])]
            target_index=label[key[1]+'-'+str(element[1])]
#             source_index=key[0]+'-'+str(element[0])
#             target_index=key[1]+'-'+str(element[1])

            value=dict_tuples[key][element]
            
            
            sources.append(source_index)
            target.append(target_index)
            values.append(value)
            
            if((element[0] in [0,1,2,99]) and (element[1] in [0,1,2,99])):
                if(key[0]=='8/17'):
                    colour_values.append(colours[i].replace("0.8", str(0.8)))
                else:
                    colour_values.append(colours[i].replace("0.8", str(0.4)))
            else:
                colour_values.append("rgba(0, 0, 0, 0.05)")
        i+=1
    return sources,target,values,colour_values,dict_tuples



def get_plotly_figure(sources,target,values,colour_values,labels_modified):
    layout = go.Layout(
      margin=go.layout.Margin(
            l=1, #left margin
            r=1, #right margin
            b=10, #bottom margin
            t=2  #top margin
        )
    )


    fig = go.Figure(data=[go.Sankey(
        arrangement = "snap",
        node = dict(
          pad = 15,
          thickness = 2,
          line = dict(color = "black", width = 0.2),
          label = labels_modified,
          color='black'
        ),
        link = dict(
          source = sources, # indices correspond to labels, eg A1, A2, A1, B1, ...
          target = target,
          value = values,
          color = colours_value
      ))],layout=layout)
    return fig

In [ ]:
import pandas as pd
from tqdm import tqdm_notebook
df_users_timeline=pd.read_csv('../Results/kcore_analysis.csv')
df_users_timeline=df_users_timeline.drop(columns=['time_to_reach','velocity'])
#df_users_timeline=df_users_timeline.replace([3,4,5,6,7,8,9],98)
# df_users_timeline['freq']=1

In [ ]:
df_users_timeline_fear=df_users_timeline[df_users_timeline['user_key'].isin(list(df_90_fear.index))]
df_users_timeline_hate=df_users_timeline[df_users_timeline['user_key'].isin(list(df_90_hate.index))]

In [ ]:
dict_tuples={}
i=8
key=tuple([months_list[i],months_list[i+1]])
dict_tuples[key]={}
wide_df = df_users_timeline_fear[[months_list[i],months_list[i+1],'freq']].groupby([months_list[i],months_list[i+1]], as_index=False).sum()


In [ ]:
months_list=list(df_users_timeline.columns)[2:-1]

label={}
count=0
for month in months_list:
    for i in range(0,10,1):
        label[month+'-'+str(i)]=count
        count+=1        
    label[month+'-'+str(99)]=count
    count+=1

labels_modified=[]
for ele in list(label.keys()):
    if ele.split('-')[1]=='99':
        labels_modified.append('NA')
    else:
        labels_modified.append(ele.split('-')[1])


In [ ]:
sources,target,values,colours_value,dict_tuples=get_sources_target_val(df_users_timeline_fear)
fig = get_plotly_figure(sources,target,values,colours_value,labels_modified)

In [ ]:
fig.write_image("../Results/sankey_fear.pdf")

In [ ]:
sources,target,values,colours_value,dict_tuples=get_sources_target_val(df_users_timeline_hate)
fig = get_plotly_figure(sources,target,values,colours_value,labels_modified)
fig.write_image("../Results/sankey_hate.pdf")

In [ ]:
import numpy as np
matplotlib.rcParams['figure.figsize'] = (3, 3)
sns.set(context='paper',style='whitegrid',font_scale=1.5)

# sns.set_theme(style="whitegrid")

# tips = sns.load_dataset("tips")
ax = sns.barplot(x="label",y="time_to_reach", data=df_90, palette=['red','orange'])
#plt.show()
path='../Results/time_to_reach.pdf'
plt.savefig(path,bbox_inches='tight',dpi=400)

In [ ]:
df_90.head()

In [ ]:
import numpy as np
matplotlib.rcParams['figure.figsize'] = (3, 3)
sns.set(context='paper',style='whitegrid',font_scale=1.5)

# sns.set_theme(style="whitegrid")

# tips = sns.load_dataset("tips")
ax = sns.barplot(x="label",y="normal_reposters", data=df_90, palette=['red','orange'])
#plt.show()
path='../Results/normal_reposters.pdf'
plt.savefig(path,bbox_inches='tight',dpi=400)

In [ ]:
import numpy as np
matplotlib.rcParams['figure.figsize'] = (3, 3)
sns.set(context='paper',style='whitegrid',font_scale=1.5)

# sns.set_theme(style="whitegrid")

# tips = sns.load_dataset("tips")
ax = sns.barplot(x="label",y="total_normal_reposts", data=df_90, palette=['red','orange'])
#plt.show()
path='../Results/total_normal_reposts.pdf'
plt.savefig(path,bbox_inches='tight',dpi=400)

In [ ]:
import numpy as np
matplotlib.rcParams['figure.figsize'] = (3, 3)
sns.set(context='paper',style='whitegrid',font_scale=1.5)

# sns.set_theme(style="whitegrid")

# tips = sns.load_dataset("tips")
ax = sns.barplot(x="label",y="normal_mentions", data=df_90, palette=['red','orange'])
#plt.show()
path='../Results/normal_mentions.pdf'
plt.savefig(path,bbox_inches='tight',dpi=400)

In [ ]:
import numpy as np
matplotlib.rcParams['figure.figsize'] = (3, 3)
sns.set(context='paper',style='whitegrid',font_scale=1.5)

# sns.set_theme(style="whitegrid")

# tips = sns.load_dataset("tips")
ax = sns.barplot(x="label",y="total_normal_mentions", data=df_90, palette=['red','orange'])
#plt.show()
path='../Results/total_normal_mentions.pdf'
plt.savefig(path,bbox_inches='tight',dpi=400)

In [ ]:
import numpy as np
matplotlib.rcParams['figure.figsize'] = (3, 3)
sns.set(context='paper',style='whitegrid',font_scale=1.5)

# sns.set_theme(style="whitegrid")

# tips = sns.load_dataset("tips")
ax = sns.barplot(x="label",y="normal_repliers", data=df_90, palette=['red','orange'])
#plt.show()
path='../Results/normal_repliers.pdf'
plt.savefig(path,bbox_inches='tight',dpi=400)

In [ ]:
import numpy as np
matplotlib.rcParams['figure.figsize'] = (3, 3)
sns.set(context='paper',style='whitegrid',font_scale=1.5)

# sns.set_theme(style="whitegrid")

# tips = sns.load_dataset("tips")
ax = sns.barplot(x="label",y="total_normal_replies", data=df_90, palette=['red','orange'])
#plt.show()
path='../Results/total_normal_repliers.pdf'
plt.savefig(path,bbox_inches='tight',dpi=400)

In [ ]:
df_90.head()

In [ ]:
from numpy.random import seed
from numpy.random import randn
from scipy.stats import mannwhitneyu
seed(1)
column_name='normal_followers'

treatment=list(df_90[(df_90['label']=='F') & (df_90[column_name]!=-1)][column_name])
control=list(df_90[(df_90['label']=='H') & (df_90[column_name]!=-1)][column_name])
print("F",np.mean(treatment))
print("H",np.mean(control))

stat, p = mannwhitneyu(treatment, control,alternative='greater')
print('Statistics=%.3f, p=%.10f' % (stat, p))

In [ ]:
top10user=list(df_90[df_90['label']=='F'].sort_values('total_normal_reposts',ascending=False)[0:10].index)

In [ ]:
import random
def get_samples(users=None,label='fearspeech'):
    threshold={'hatespeech':0.9,'fearspeech':0.7,'normal':0.7}
    posts=[]
    files = sorted(glob('../../Gab_Data/new_features_old_gab/gab_fear_hate_features*.pickle'))
    for file in tqdm_notebook(files,total=len(files)):
   
        with open(file, 'rb') as handle:
            Gab_keyword_match = pickle.load(handle)

        if('predicted_probab' not in Gab_keyword_match[1].keys()):
            print("not found")
            continue

        for element in Gab_keyword_match:
            
            post={}
            post['body']=element['post_body']
            labels=[]
    
            for i in range(len(element['predicted_probab'])):
                if(element['predicted_probab'][i]>threshold[reverse_dict_labels[i]]):
                    labels.append(reverse_dict_labels[i])

            post['labels']=labels
            
            if(label=='normal' and post['labels']==[]):
                if(element['username'] in users):
                    #print(line,labels)
                    posts.append([element['post_body'],'normal',element['like_count'],element['repost_count']])
            
            if(post['labels']==[label]):
                if(element['username'] in users):
#                     #print(line,labels)
                    posts.append([element['post_body'],labels[0],element['like_count'],element['repost_count'],element['reply_count']])
    print(len(posts))
    df=pd.DataFrame(posts, columns=['text','label','like','repost','reply'])
    return df
#     print(len(posts))
#     samples=random.sample(posts,min(len(posts),20))
    
#     for sample in samples:
#         print(sample)
#         print("==============================================")

def get_samples_date(month_start=1,month_end=3,year_start=2017,year_end=2017,label='fearspeech'):
    threshold={'hatespeech':0.9,'fearspeech':0.7,'normal':0.7}
    posts=[]
    files = sorted(glob('../../Gab_Data/new_features_old_gab/gab_fear_hate_features*.pickle'))
    for file in tqdm_notebook(files,total=len(files)):
   
        with open(file, 'rb') as handle:
            Gab_keyword_match = pickle.load(handle)

        if('predicted_probab' not in Gab_keyword_match[1].keys()):
            print("not found")
            continue

        for element in Gab_keyword_match:
            flag_add=False
            try:
                dt = datetime.fromisoformat(element['post_create_time'])
                if((dt.month >= month_start and dt.month <= month_end) and (dt.year >= year_start and dt.year <= year_end)):
                    flag_add=True
                else:
                    flag_add=False
#                 key_new=str(dt.month)+'/'+str(dt.year)
            except KeyError:
                continue
            
            
            post={}
            post['body']=element['post_body']
            labels=[]
            
            
            
            for i in range(len(element['predicted_probab'])):
                if(element['predicted_probab'][i]>threshold[reverse_dict_labels[i]]):
                    labels.append(reverse_dict_labels[i])

            post['labels']=labels
            
            if(label=='normal' and post['labels']==[]):
#                 if(element['username'] in users):
#                     #print(line,labels)
                    if(flag_add):
                        posts.append([element['post_body'],'normal',element['like_count'],element['repost_count']])
            
            if(post['labels']==[label]):
#                 if(element['username'] in users):
#                     #print(line,labels)
                    if(flag_add):
                        posts.append([element['post_body'],labels[0],element['like_count'],element['repost_count'],element['reply_count']])
    print(len(posts))
    df=pd.DataFrame(posts, columns=['text','label','like','repost','reply'])
    return df
#     print(len(posts))
#     samples=random.sample(posts,min(len(posts),20))
    
#     for sample in samples:
#         print(sample)
#         print("==============================================")



In [ ]:
df=get_samples(users=None,label='fearspeech')

In [ ]:
df.sort_values('reply',ascending=False)[0:20]

In [ ]:
df_mentions=get_samples_date(month_start=2,month_end=3,year_start=2018,year_end=2018,label='hatespeech')

In [ ]:
df_mentions_hate=get_samples_date(month_start=2,month_end=3,year_start=2018,year_end=2018,label='hatespeech')

In [ ]:
df_mentions_hate.sort_values('repost',ascending=False)[0:100]

In [ ]:
mentioned_dict_highest={}
for index,row in df_mentions.iterrows():
    text=row['text'].split()
    for word in text:
        mentioned_user=None
        if(len(word.split('@'))>1):
            res = word.split('@')[1]

            # TO AVOID GMAILS
            if(len(res.split('.'))>1):
                if_gmail=res.split('.')[1]
                if(if_gmail!='com'):
                    mentioned_user=res
            else:
                mentioned_user=res
        
        try:
            mentioned_dict_highest[mentioned_user]+=1
        except KeyError:
            mentioned_dict_highest[mentioned_user]=1

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    for ele in list(df_mentions.sample(10).text):
        print(ele)
        print("------------")

In [ ]:
df_hate=get_samples(users=['JaneDoeCountry', 'st3pp3nw0lf', 'Sterangeli5', 'WarRoomShow', 'Lynne75', 'Castelnau', 'Kaybird', 'Pamisue', 'LesaJoy', 'Didizichi'],label='normal')

In [ ]:
df_hate.to_csv('mentioned_nromal.csv')

In [ ]:
pd.set_option('display.max_colwidth', -1)
df.sort_values(by='like',ascending=False).head(20)


In [ ]:
import json
with open('../../../../Gab_Data_old/user_details.json') as fp:
    dict_users_original = json.load(fp)

exfearuser=list(df_90[df_90['label']=='F'].sort_values('total_normal_reposts',ascending=False).index)
def get_mentions(text):
    mentioned_users=[]
    text=text.split()
    for word in text:
        muser=None
        if(len(word.split('@'))>1):
            res = word.split('@')[1]
            # TO AVOID GMAILS
            if(len(res.split('.'))>1):
                if_gmail=res.split('.')[1]
                if(if_gmail!='com'):
                    muser=res
            else:
                muser=res

                    
            if(muser!=None):
                try:
                    temp=dict_users_original[muser]
                    mentioned_users.append(muser)
                except KeyError:
                    continue
    return mentioned_users
        
        
def get_mentioned_users(users=exfearuser,label='fearspeech'):
    threshold={'hatespeech':0.9,'fearspeech':0.7,'normal':0.7}
    posts=[]
    mentioned_user_dict={}
    files = sorted(glob('../../Gab_Data/new_features_old_gab/gab_fear_hate_features*.pickle'))
    for file in tqdm_notebook(files,total=len(files)):
   
        with open(file, 'rb') as handle:
            Gab_keyword_match = pickle.load(handle)

        if('predicted_probab' not in Gab_keyword_match[1].keys()):
            print("not found")
            continue

        for element in Gab_keyword_match:
            
            post={}
            post['body']=element['post_body']
            
            
            labels=[]
    
            for i in range(len(element['predicted_probab'])):
                if(element['predicted_probab'][i]>threshold[reverse_dict_labels[i]]):
                    labels.append(reverse_dict_labels[i])

            post['labels']=labels
            
            if(label=='normal' and post['labels']==[]):
                if(element['username'] in users):
                    #print(line,labels)
                    posts.append([element['post_body'],'normal',element['like_count'],element['repost_count']])
            
            if(post['labels']==[label]):
                if(element['username'] in users):
                    #print(line,labels)
                    
                    mentioned_users=get_mentions(element['post_body'])
                    
                    
                    for element in mentioned_users:
                        try:
                            temp=all_normal_dict[element]
                            try:
                                mentioned_user_dict[element]+=1
                            except KeyError:
                                mentioned_user_dict[element]=1
                        except:
                            pass
                    
                                
                    
                    #posts.append([element['post_body'],labels[0],element['like_count'],element['repost_count']])
    return mentioned_user_dict

### Hybrid Users

In [ ]:
mentioned_user_dict=get_mentioned_users(users=exfearuser,label='fearspeech')

In [ ]:
sorted_dict={k: v for k, v in sorted(mentioned_user_dict.items(), key=lambda item: item[1], reverse=True)[0:10]}
sorted_dict.keys()

In [ ]:
avg_follower=0
avg_following=0 
avg_post=0
for key in sorted_dict.keys():
    if(key=='WarRoomShow'):
        continue
    
    avg_follower+=dict_users_original[key]['follower_count']/10
    avg_following+=dict_users_original[key]['following_count']/10
    avg_post+=dict_users_original[key]['post_count']/10
    
    

In [ ]:
print(avg_follower,avg_following,avg_post)

### Temporal analysis

In [ ]:
interaction_dict={}
exfearuser=list(df_90[df_90['label']=='F'].sort_values('total_normal_reposts',ascending=False).index)
exhateuser=list(df_90[df_90['label']=='H'].sort_values('total_normal_reposts',ascending=False).index)


def check_condition(users,key,user_data,type_interaction='replies'):
    if(type_interaction=='replies'):
        if(key[1] in users) and (key[0] in all_normal_dict) and user_data[key][type_interaction]>0:
            return True
        else:
            return False
    elif(type_interaction=='reposts'):
        if(key[0] in users) and (key[1] in all_normal_dict) and user_data[key][type_interaction]>0:
            return True
        else:
            return False
    elif(type_interaction=='mentions'):
        if(key[0] in users) and (key[1] in all_normal_dict) and user_data[key][type_interaction]>0:
            return True
        else:
            return False


        
def get_interaction_dict(interaction_data,users,type_interaction):
    interaction_dict={}
    for month_year in tqdm_notebook(interaction_data.keys()):
        user_data=interaction_data[month_year]
        user_replied={}
        for key in user_data.keys():

            if(check_condition(users,key,user_data,type_interaction=type_interaction)):

                try:

                    interaction_dict[month_year]['total_interaction']+=user_data[key][type_interaction]
                    interaction_dict[month_year]['total_users']+=1

                    try:
                        temp=user_replied[key[0]]
                    except KeyError:
                        user_replied[key[0]]=1
                        interaction_dict[month_year]['unique_users']+=1

                except KeyError:
                    interaction_dict[month_year]={}
                    interaction_dict[month_year]['total_interaction']=user_data[key][type_interaction]
                    interaction_dict[month_year]['total_users']=1


                    try:
                        temp=user_replied[key[0]]
                    except KeyError:
                        user_replied[key[0]]=1
                        interaction_dict[month_year]['unique_users']=1
    return interaction_dict

In [ ]:
# interaction_dict_exfear_reply=get_interaction_dict(interaction_data,exfearuser,'replies')
# interaction_dict_exhate_reply=get_interaction_dict(interaction_data,exhateuser,'replies')
# interaction_dict_exfear_repost=get_interaction_dict(interaction_data,exfearuser,'reposts')
# interaction_dict_exhate_repost=get_interaction_dict(interaction_data,exhateuser,'reposts')
interaction_dict_exfear_mention=get_interaction_dict(interaction_data,exfearuser,'mentions')
interaction_dict_exhate_mention=get_interaction_dict(interaction_data,exhateuser,'mentions')


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.gridspec as gridspec
matplotlib.rcParams['figure.figsize'] = (20, 5)
sns.set(context='paper',style='whitegrid',font_scale=2, rc={"lines.linewidth": 2.0,"lines.markersize": 10})

x=[]
y=[]

for key in interaction_dict_exfear_mention:
    x.append(key[0:2]+'-'+key[-2:])
    y.append(interaction_dict_exfear_mention[key]['total_interaction']/interaction_dict_exfear_mention[key]['unique_users'])

sns.lineplot(x=x, y=y, color='red')

x=[]
y=[]

for key in interaction_dict_exhate_mention:
    x.append(key[0:2]+'-'+key[-2:])
    y.append(interaction_dict_exhate_mention[key]['total_interaction']/interaction_dict_exhate_mention[key]['unique_users'])

sns.lineplot(x=x, y=y,color='orange')    

In [ ]:
interaction_dict_exhate_mention[key]['unique_users']


### Get fractions replied to by normal users

In [ ]:
files = sorted(glob('../../Gab_Data/new_features_old_gab/gab_fear_hate_features*.pickle'))

#dict_posts_ids={}
for file in tqdm_notebook(files,total=len(files)):
    with open(file, 'rb') as handle:
        Gab_keyword_match = pickle.load(handle)
        
    if('predicted_probab' not in Gab_keyword_match[1].keys()):
        print("not found")
        continue
        
    for element in Gab_keyword_match:
        post={}
        post['body']=element['post_body']


        labels=[]

        for i in range(len(element['predicted_probab'])):
            if(element['predicted_probab'][i]>threshold[reverse_dict_labels[i]]):
                labels.append(reverse_dict_labels[i])

        post['labels']=labels

        if(label=='normal' and post['labels']==[]):
            pass
#         elif(post['labels']==['fearspeech'] or post['labels']==['hatespeech']):
#             dict_posts_ids[element['id']]['label']=post['labels'][0]
        


        

In [ ]:
len(dict_post_ids)